## HW2: Linear Discriminant Analysis
In hw2, you need to implement Fisher’s linear discriminant by using only numpy, then train your implemented model by the provided dataset and test the performance with testing data

Please note that only **NUMPY** can be used to implement your model, you will get no points by calling sklearn.discriminant_analysis.LinearDiscriminantAnalysis 

## Load data

In [4]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score

In [5]:
x_train = np.load("x_train.npy")
y_train = np.load("y_train.npy")
x_test = np.load("x_test.npy")
y_test = np.load("y_test.npy")

In [6]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(750, 2)
(750,)
(250, 2)
(250,)


## 1. Compute the mean vectors mi, (i=1,2) of each 2 classes

In [19]:
## Your code HERE
m1 = x_train[y_train == 0].mean(0)
m2 = x_train[y_train == 1].mean(0)

In [20]:
print(f"mean vector of class 1: {m1}", f"mean vector of class 2: {m2}")

mean vector of class 1: [ 1.3559426  -1.34746216] mean vector of class 2: [-1.29735587  1.29096203]


## 2. Compute the Within-class scatter matrix SW

In [21]:
## Your code HERE
c1 = x_train[y_train == 0]
c2 = x_train[y_train == 1]
sw1 = []
for i in range(len(c1)):
    sw1 += [np.dot((c1[i] - m1).reshape(2, 1), (c1[i] - m1).reshape(2, 1).T)]
sw1 = sum(sw1)
sw2 = []
for i in range(len(c2)):
    sw2 += [np.dot((c2[i] - m2).reshape(2, 1), (c2[i] - m2).reshape(2, 1).T)]
sw2 = sum(sw2)
sw = sw1 + sw2

In [22]:
assert sw.shape == (2, 2)
print(f"Within-class scatter matrix SW: {sw}")

Within-class scatter matrix SW: [[ 388.64001349 -228.92177708]
 [-228.92177708  665.56910433]]


## 3.  Compute the Between-class scatter matrix SB

In [23]:
## Your code HERE
sb = np.dot((m2 - m1).reshape(2, 1), (m2 - m1).reshape(2, 1).T)

In [24]:
assert sb.shape == (2, 2)
print(f"Between-class scatter matrix SB: {sb}")

Between-class scatter matrix SB: [[ 7.03999279 -7.00052687]
 [-7.00052687  6.9612822 ]]


## 4. Compute the Fisher’s linear discriminant

In [25]:
## Your code HERE
eigval, eigvec = np.linalg.eig(np.dot(np.linalg.inv(sw), sb))
idx = eigval.argsort()[::-1]
w = eigvec[:, idx][:, :1]

In [26]:
assert w.shape == (2, 1)
print(f" Fisher’s linear discriminant: {w}")

 Fisher’s linear discriminant: [[ 0.94096648]
 [-0.33849976]]


## 5. Project the test data by linear discriminant to get the class prediction by nearest-neighbor rule and calculate the accuracy score 
you can use accuracy_score function from sklearn.metric.accuracy_score

In [30]:
# Predict y by K-nearest-neighbor
K = 1
train = np.matmul(x_train, w)
test = np.matmul(x_test, w)
y_pred = []

for i in range(len(test)):
    dists = np.zeros(len(train))
    for j in range(len(train)):
        dists[j] = np.sqrt(np.sum((test[i] - train[j]) ** 2))
    dists = dists.argsort()
    neighbor = list(y_train[dists][:K])
    y_pred.append(max(set(neighbor), key=neighbor.count))

In [31]:
acc = accuracy_score(y_test, y_pred)

In [32]:
print(f"Accuracy of test-set {acc}")

Accuracy of test-set 0.916


## 6. Plot the 1) best projection line on the training data and show the slope and intercept on the title (you can choose any value of intercept for better visualization) 2) colorize the data with each class 3) project all data points on your projection line. Your result should look like [this image](https://i.imgur.com/tubMQpw.jpg)

In [ ]:
x = np.linspace(-2, 2, 30)
slope = w[1] / w[0]
intercept = 0
line_function = np.poly1d([slope, intercept])
y = line_function(x)



plt.figure(figsize=(10, 10))
plt.xlim(-2, 1)
plt.ylim(-2, 1)
plt.axis('equal')
plt.title(f"Project Line: slope={slope[0]}, intercept={intercept}")

x_dot, y_dot = x_train[y_train == 0][:, 0], x_train[y_train == 0][:, 1]
plt.plot(x_dot, y_dot, 'ro', color='red', label='class1')
x_dot, y_dot = x_train[y_train == 1][:, 0], x_train[y_train == 1][:, 1]
plt.plot(x_dot, y_dot, 'ro', color='blue', label='class2')


plt.plot(x, y, c='black')
for i in range(len(x_train)):
    x, y = x_train[i][0], x_train[i][1]
    proj_x = ((x * (1 / slope[0]) + y) - intercept) / (slope[0] + 1 / slope[0])
    proj_y = slope[0] * proj_x + intercept
    x_dot, y_dot = [x_train[i][0], proj_x], [x_train[i][1], proj_y]
    if(y_train[i] == 0):
        plt.scatter(proj_x, proj_y, s=8, c='red')
        plt.plot(x_dot, y_dot, c='yellow', linewidth=0.1)
    else:
        plt.scatter(proj_x, proj_y, s=8, c='blue')
        plt.plot(x_dot, y_dot, c='green', linewidth=0.1)

plt.show()